In [2]:
from subsbml import System, createSubsystem, combineSystems, createNewSubsystem
import numpy as np
import pylab as plt
import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

Loading BokehJS ...

# cell 1

In [3]:
ss1 = createSubsystem('fusion_sub1.xml')
ss3 = createSubsystem('atc_external_reservoir.xml')
mb1 = createSubsystem('membrane_atc.xml', membrane = True)
cell_1 =  System('cell_1',ListOfInternalSubsystems = [ss1],
                 ListOfExternalSubsystems = [ss3],
                 ListOfMembraneSubsystems = [])

cell_1_model = cell_1.getModel()
cell_1_model.setSpeciesAmount('atc_e', 1e4, compartment = 'cell_1_external')
cell_1_model.setSpeciesAmount('dna_f', 1, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('tetR', 10, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('ATP', 20, compartment = 'cell_1_internal')


The subsystem from membrane_atc.xml has multiple compartments


True

# cell 2

In [4]:
ss2 = createSubsystem('fusion_sub2.xml')
ss3 = createSubsystem('atc_external_reservoir.xml')
mb1 = createSubsystem('membrane_atc.xml', membrane = True)
cell_2 =  System('cell_2',ListOfInternalSubsystems = [ss2],
                 ListOfExternalSubsystems = [ss3],
                 ListOfMembraneSubsystems = [mb1])
cell_2_model = cell_2.getModel()
cell_2_model.setSpeciesAmount('atc_e', 1e4, compartment = 'cell_2_external')
cell_2_model.setSpeciesAmount('dna_h', 1, compartment = 'cell_2_internal')
cell_2_model.setSpeciesAmount('ATP', 20, compartment = 'cell_2_internal')


The subsystem from membrane_atc.xml has multiple compartments


True

# prep. for fusing by changing compartment

In [5]:
cell_1_model.renameCompartments('cell_1_internal', 'cellfusedinternal')
cell_1_model.renameCompartments('cell_1_external', 'cellfusedexternal')
cell_2_model.renameCompartments('cell_2_internal', 'cellfusedinternal')
cell_2_model.renameCompartments('cell_2_external', 'cellfusedexternal')
cell_1_model.writeSBML('m2_cell_1_model.xml')
cell_2_model.writeSBML('m2_cell_2_model.xml')

1

# fusing subsystem

In [6]:
fus_comb=createNewSubsystem()
fus_comb.combineSubsystems([cell_1_model,cell_2_model], verbose = True)

_ = fus_comb.writeSBML('m2_fus_comb.xml')

The Ids are identical, renaming stopped.


C:\Users\Agrima\anaconda3\lib\site-packages\subsbml-0.1-py3.7.egg\subsbml\Subsystem.py:646: UserWarning: Not all of the Subsystems being combined are in the same Compartment
C:\Users\Agrima\anaconda3\lib\site-packages\subsbml-0.1-py3.7.egg\subsbml\Subsystem.py:989: UserWarning: Annotated propensity updated to: <PropensityType> type=massaction k=2e-05  </PropensityType>
C:\Users\Agrima\anaconda3\lib\site-packages\subsbml-0.1-py3.7.egg\subsbml\Subsystem.py:989: UserWarning: Annotated propensity updated to: <PropensityType> type=massaction k=2e-09  </PropensityType>
C:\Users\Agrima\anaconda3\lib\site-packages\subsbml-0.1-py3.7.egg\subsbml\Subsystem.py:989: UserWarning: Annotated propensity updated to: <PropensityType> type=massaction k=4e-07  </PropensityType>
C:\Users\Agrima\anaconda3\lib\site-packages\subsbml-0.1-py3.7.egg\subsbml\Subsystem.py:989: UserWarning: Annotated propensity updated to: <PropensityType> type=massaction k=1e-12  </PropensityType>
C:\Users\Agrima\anaconda3\lib\site

In [10]:
timepoints = np.linspace(0,25000,10000)
results_1, _ = cell_1_model.simulateWithBioscrape(timepoints)
results_2, _ = cell_2_model.simulateWithBioscrape(timepoints)
results_3, _ = fus_comb.simulateWithBioscrape(timepoints)
exsA_id1 = cell_1_model.getSpeciesByName('exsA', compartment = 'cellfusedinternal').getId()
gfp_id2 = cell_2_model.getSpeciesByName('gfp', compartment = 'cellfusedinternal').getId()
gfp_id3 = fus_comb.getSpeciesByName('gfp', compartment = 'cellfusedinternal').getId()
exsA_id2 = fus_comb.getSpeciesByName('exsA', compartment = 'cellfusedinternal').getId()

colors=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d']
fused_plot = bokeh.plotting.figure(width = 800, height = 450, 
                         x_axis_label = 'time(sec)',
                         y_axis_label = 'concentration',
                         title = 'Fusion Plot')
fused_plot.line(timepoints, results_1[exsA_id1], color = "blue", line_width = 3, legend_label = 'exsA_cell1')
fused_plot.line(timepoints, results_2[gfp_id2], color = "cyan",line_width = 3, legend_label = 'gfp_cell2')
fused_plot.line(timepoints, results_3[gfp_id3], color = "green",line_width = 3, legend_label = 'gfp_fusedcell')
fused_plot.line(timepoints, results_3[exsA_id2], color = "red", line_width = 3,legend_label = 'exsA_fusedcell')
fused_plot.legend.location = 'center_right'
fused_plot.legend.click_policy="hide"

bokeh.io.show(fused_plot)


In [ ]:
results_3